In [2]:
import os
import json
from elasticsearch import Elasticsearch

from pprint import pprint

client = Elasticsearch(
    hosts=["http://localhost:9200"],
    api_key="VHFpUXlaVUI1YWl6WDY0T3BxUnM6VGpoWUwtSzNTdkdCYXhPWkx5c1ZSUQ==",
)

resp = client.ping()
print(resp)

def output(data):
    json_data = json.dumps(data.body, indent=4)
    print(json_data)



True


In [3]:
basic_index="my-index"
if not client.indices.exists(index=basic_index):
    resp = client.indices.create(
       index="my-index",
    )
else:
    print(f'Index {basic_index} already exists')


Index my-index already exists


In [4]:
e5_inference="my_e5_model"
if not client.inference.get(inference_id=e5_inference):
    resp = client.inference.put(
        task_type=e5_inference,
        inference_id="my_e5_model",
        inference_config={
            "service": "elasticsearch",
            "service_settings": {
                "num_threads": 1,
                "model_id": ".multilingual-e5-small",
                "adaptive_allocations": {
                    "enabled": False
                }
            }
        },
    )
    output(resp)
else:
    print(f'Inference {e5_inference} already exists')

Inference my_e5_model already exists


## Test out the inference service

In [5]:

resp = client.inference.inference(
    task_type="text_embedding",
    inference_id="my_e5_model",
    input="my awesome piece of text",
)

output(resp)


{
    "text_embedding": [
        {
            "embedding": [
                0.017882703,
                0.0075477664,
                -0.051404208,
                -0.07948811,
                0.07463727,
                -0.033370923,
                0.053825323,
                0.037787367,
                0.006766823,
                0.008241204,
                0.010106811,
                0.033864826,
                0.08292517,
                -0.019575875,
                -0.058219235,
                0.04311379,
                0.083135895,
                -0.07104062,
                -0.064723305,
                -0.029838856,
                0.02827,
                0.014934688,
                -0.03497228,
                0.019852914,
                0.044561114,
                0.041011676,
                -0.05760241,
                0.0032730384,
                0.010831467,
                -0.06961988,
                -0.06764084,
                -0.021722091,
       

In [6]:
bbq_index = "bbq-my-byte-quantized-index"
if not client.indices.exists(index=bbq_index):
    resp = client.indices.create(
        index="bbq-my-byte-quantized-index",
        mappings={
            "properties": {
                "my_field": {
                    "type": "text"
                },
                "my_vector": {
                    "type": "dense_vector",
                    "dims": 384,
                    "index_options": {
                        "type": "bbq_hnsw"
                    }
                }
            }
        },
    )
    output(resp)

else:
    print(f'The index {bbq_index} already exists.')


The index bbq-my-byte-quantized-index already exists.


In [7]:

resp = client.ingest.put_pipeline(
    id="my_inference_pipeline",
    processors=[
        {
            "inference": {
                "model_id": "my_e5_model",
                "input_output": [
                    {
                        "input_field": "my_field",
                        "output_field": "my_vector"
                    }
                ]
            }
        }
    ],
)

output(resp)



{
    "acknowledged": true
}


In [8]:

resp = client.ingest.simulate(
    id="my_inference_pipeline",
    docs=[
        {
            "_source": {
                "my_field": "my awesome text field"
            }
        }
    ],
)

output(resp)



{
    "docs": [
        {
            "doc": {
                "_index": "_index",
                "_version": "-3",
                "_id": "_id",
                "_source": {
                    "my_field": "my awesome text field",
                    "model_id": "my_e5_model",
                    "my_vector": [
                        0.014117197133600712,
                        0.016865601763129234,
                        -0.008850697427988052,
                        -0.09943144768476486,
                        0.07708954066038132,
                        -0.05163203552365303,
                        0.031171943992376328,
                        0.04832464084029198,
                        -0.020899513736367226,
                        -0.01805395446717739,
                        0.027316145598888397,
                        0.014158275909721851,
                        0.11380597949028015,
                        -0.015239506028592587,
                        -0.06438340246677

In [9]:
resp = client.index(
    index="bbq-my-byte-quantized-index",
    id="1",
    pipeline="my_inference_pipeline",
    document={
        "my_field": "my awesome text field"
    },
)

output(resp)



{
    "_index": "bbq-my-byte-quantized-index",
    "_id": "1",
    "_version": 6,
    "result": "updated",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 7,
    "_primary_term": 1
}


In [10]:
resp = client.index(
    index="bbq-my-byte-quantized-index",
    id="2",
    pipeline="my_inference_pipeline",
    document={
        "my_field": "some other sentence"
    },
)

output(resp)



{
    "_index": "bbq-my-byte-quantized-index",
    "_id": "2",
    "_version": 3,
    "result": "updated",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 8,
    "_primary_term": 1
}


In [11]:
resp = client.search(
    index="bbq-my-byte-quantized-index",
    query={
        "bool": {
            "must": [
                {
                    "knn": {
                        "field": "my_vector",
                        "query_vector_builder": {
                            "text_embedding": {
                                "model_id": "my_e5_model",
                                "model_text": "my awesome search field"
                            }
                        },
                        "k": 10,
                        "num_candidates": 100
                    }
                }
            ]
        }
    },
    source=[
        "my_field"
    ],
)

output(resp)



{
    "took": 16,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 2,
            "relation": "eq"
        },
        "max_score": 0.9632075,
        "hits": [
            {
                "_index": "bbq-my-byte-quantized-index",
                "_id": "1",
                "_score": 0.9632075,
                "_source": {
                    "my_field": "my awesome text field"
                }
            },
            {
                "_index": "bbq-my-byte-quantized-index",
                "_id": "2",
                "_score": 0.89579916,
                "_source": {
                    "my_field": "some other sentence"
                }
            }
        ]
    }
}


### Now lets observe standard hnsw:

In [63]:
standard_index = "my-raw-vector-index"
if not client.indices.exists(index=standard_index):
    resp = client.indices.create(
        index="my-raw-vector-index",
        mappings={
            "properties": {
                "my_field": {
                    "type": "text"
                },
                "my_vector": {
                    "type": "dense_vector",
                    "dims": 384,
                    "index_options": {
                        "type": "hnsw"
                    }
                }
            }
        },
    )
    output(resp)

else: 
    print(f'Index {standard_index} already exists.')


Index my-raw-vector-index already exists.


In [64]:
resp = client.index(
    index="my-raw-vector-index",
    id="1",
    pipeline="my_inference_pipeline",
    document={
        "my_field": "my awesome text field"
    },
)

output(resp)



{
    "_index": "my-raw-vector-index",
    "_id": "1",
    "_version": 2,
    "result": "updated",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 2,
    "_primary_term": 1
}


In [65]:
resp = client.index(
    index="my-raw-vector-index",
    id="2",
    pipeline="my_inference_pipeline",
    document={
        "my_field": "some other sentence"
    },
)

output(resp)



{
    "_index": "my-raw-vector-index",
    "_id": "2",
    "_version": 2,
    "result": "updated",
    "_shards": {
        "total": 2,
        "successful": 1,
        "failed": 0
    },
    "_seq_no": 3,
    "_primary_term": 1
}


In [67]:
resp = client.search(
    index="my-raw-vector-index",
    query={
        "bool": {
            "must": [
                {
                    "knn": {
                        "field": "my_vector",
                        "query_vector_builder": {
                            "text_embedding": {
                                "model_id": "my_e5_model",
                                "model_text": "my awesome search field"
                            }
                        },
                        "k": 10,
                        "num_candidates": 100
                    }
                }
            ]
        }
    },
    source=[
        "my_field"
    ],
)

output(resp)



{
    "took": 6,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 2,
            "relation": "eq"
        },
        "max_score": 0.9632075,
        "hits": [
            {
                "_index": "my-raw-vector-index",
                "_id": "1",
                "_score": 0.9632075,
                "_source": {
                    "my_field": "my awesome text field"
                }
            },
            {
                "_index": "my-raw-vector-index",
                "_id": "2",
                "_score": 0.89579916,
                "_source": {
                    "my_field": "some other sentence"
                }
            }
        ]
    }
}
